In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
import os
import pandas as pd
import pickle
import re

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/01_level1')
from level1_utils import run_level1, make_level1_design_matrix, get_events

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'

subnum = '01'
runnum='4'

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
# plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
mnum = 'model1'
reg_rt = 1
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt")%(mnum, str(reg_rt)

# run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt = reg_rt)

In [ ]:
mnum = 'model2'
reg_rt = 0
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt")%(mnum, str(reg_rt)
                                                                                
# run_level1(subnum, 'model2', data_path, behavior_path, out_path, regress_rt = 0)

Function to get average correlation in design matrices

In [8]:
def get_mean_desmat_cor(mnum, reg_rt, l1_path):

    des_mats = glob.glob(os.path.join(l1_path, 'model%s_reg-rt%s/**/*design_matrix*'%(mnum, reg_rt)))


    beh_regs = pd.read_csv(des_mats[0]).columns
    to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
    beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]

    cor_mats = []
    for i, cur_des_mat in enumerate(des_mats):
        cur_des_mat = pd.read_csv(cur_des_mat)
        cor_mats.append(cur_des_mat[beh_regs].corr())

    df_concat = pd.concat(cor_mats)
    by_row_index = df_concat.groupby(df_concat.index)
    df_means = by_row_index.mean()
    
    return df_means


In [9]:
mnum = "1"
reg_rt = "1"
l1_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/")

get_mean_desmat_cor(mnum, reg_rt, l1_path)


,choice_st,cross_ev,cross_rt,fractalProb_ev,reward_ev,stim_ev,stim_rt
choice_st,1.000000,-0.727746,-0.204338,0.883895,0.399411,0.970985,0.001728
cross_ev,-0.727746,1.000000,0.005791,-0.507969,-0.633976,-0.788727,-0.065788
cross_rt,-0.204338,0.005791,1.000000,-0.245118,-0.103321,-0.181062,-0.031689
fractalProb_ev,0.883895,-0.507969,-0.245118,1.000000,0.009968,0.754971,0.002497
reward_ev,0.399411,-0.633976,-0.103321,0.009968,1.000000,0.584014,-0.092207
stim_ev,0.970985,-0.788727,-0.181062,0.754971,0.584014,1.000000,0.002369
stim_rt,0.001728,-0.065788,-0.031689,0.002497,-0.092207,0.002369,1.000000


In [10]:
mnum = "2"
reg_rt = "0"
l1_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/")

get_mean_desmat_cor(mnum, reg_rt, l1_path)

,choice_st,cross_ev,fractalProb_ev,fractalProb_par,reward_ev,stim_ev
choice_st,1.000000,-0.794875,0.883895,0.660567,0.399411,0.851919
cross_ev,-0.794875,1.000000,-0.545384,-0.401978,-0.709274,-0.802301
fractalProb_ev,0.883895,-0.545384,1.000000,0.747862,0.009968,0.651633
fractalProb_par,0.660567,-0.401978,0.747862,1.000000,0.024525,0.428105
reward_ev,0.399411,-0.709274,0.009968,0.024525,1.000000,0.502127
stim_ev,0.851919,-0.802301,0.651633,0.428105,0.502127,1.000000
